# Setup

In [50]:
import os
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim

In [51]:
from metric_sets import metrics_for_correlation_analysis_with_kappa

In [52]:
report_output_root_dir = Path('../../output')

report_output_dirs = { 'binary_balanced': report_output_root_dir / 'binary_balanced',
                       'binary_unbalanced': report_output_root_dir / 'binary_unbalanced',
                       
                       'multiclass_balanced': report_output_root_dir / 'multiclass_balanced',          
                       'multiclass_unbalanced': report_output_root_dir / 'multiclass_unbalanced',
                       
                       'multilabel_balanced': report_output_root_dir / 'multilabel_balanced',
                       'multilabel_unbalanced': report_output_root_dir / 'multilabel_unbalanced'}

# Get data

In [62]:
task_type = "multiclass_unbalanced"
dataset = "fma_small"
# models, indexes can be different

In [63]:
path = report_output_dirs[task_type]
print(path)

..\..\output\multiclass_unbalanced


In [64]:
json_files = list(path.rglob('*.json'))
print(json_files)

[WindowsPath('../../output/multiclass_unbalanced/sus.json'), WindowsPath('../../output/multiclass_unbalanced/cnn_fma_small_1/1.json'), WindowsPath('../../output/multiclass_unbalanced/cnn_fma_small_2/1.json'), WindowsPath('../../output/multiclass_unbalanced/cnn_fma_small_3/1.json'), WindowsPath('../../output/multiclass_unbalanced/cnn_fma_small_4/1.json'), WindowsPath('../../output/multiclass_unbalanced/cnn_fma_small_5/1.json')]


In [65]:
print(len(json_files))

6


In [66]:
import json

filtered_json_files = []
for file in json_files:
    with open(file, 'r') as f:
        data = json.load(f)
        if data.get("dataset name") == dataset:
            filtered_json_files.append(file)

dataset_json_files = filtered_json_files
print(dataset_json_files)

[WindowsPath('../../output/multiclass_unbalanced/cnn_fma_small_1/1.json'), WindowsPath('../../output/multiclass_unbalanced/cnn_fma_small_2/1.json'), WindowsPath('../../output/multiclass_unbalanced/cnn_fma_small_3/1.json'), WindowsPath('../../output/multiclass_unbalanced/cnn_fma_small_4/1.json'), WindowsPath('../../output/multiclass_unbalanced/cnn_fma_small_5/1.json')]


In [67]:
print(len(dataset_json_files))

5


# Calculate matrixes

In [68]:
import json

all_metrics = []
for file in dataset_json_files:
    with open(file, 'r') as f:
        data = json.load(f)
        all_metrics.append(data['metrics'])
print(all_metrics)

[{'macro_accuracy': 0.8503124713897705, 'micro_accuracy': 0.8503124713897705, 'accuracy_per_class': {'Electronic': 0.8787500262260437, 'Experimental': 0.8237500190734863, 'Folk': 0.8399999737739563, 'Hip-Hop': 0.8262500166893005, 'Instrumental': 0.856249988079071, 'International': 0.8231250047683716, 'Pop': 0.8487499952316284, 'Rock': 0.9056249856948853}, 'macro_precision': 0.4243984818458557, 'micro_precision': 0.4012500047683716, 'precision_per_class': {'Electronic': 0.5214285850524902, 'Experimental': 0.3136363625526428, 'Folk': 0.38429751992225647, 'Hip-Hop': 0.40441176295280457, 'Instrumental': 0.39436620473861694, 'International': 0.35836178064346313, 'Pop': 0.21621622145175934, 'Rock': 0.8024691343307495}, 'macro_recall': 0.4012499749660492, 'micro_recall': 0.4012500047683716, 'recall_per_class': {'Electronic': 0.36500000953674316, 'Experimental': 0.3449999988079071, 'Folk': 0.4650000035762787, 'Hip-Hop': 0.824999988079071, 'Instrumental': 0.2800000011920929, 'International': 0.

In [69]:
selected_metrics = [
    {k: m[k] for k in metrics_for_correlation_analysis_with_kappa if k in m}
    for m in all_metrics
]
print(selected_metrics)

[{'macro_accuracy': 0.8503124713897705, 'micro_accuracy': 0.8503124713897705, 'macro_precision': 0.4243984818458557, 'micro_precision': 0.4012500047683716, 'macro_recall': 0.4012499749660492, 'micro_recall': 0.4012500047683716, 'macro_f1': 0.3818039298057556, 'micro_f1': 0.4012500047683716, 'Kappa': 0.3157142996788025, 'MSE': 0.09347165375947952, 'LogLoss': 1.6848804950714111, 'AUNU': 0.7977339285714287, 'micro_ROC-AUC': 0.7907915457589285}, {'macro_accuracy': 0.8414062261581421, 'micro_accuracy': 0.8414062261581421, 'macro_precision': 0.43838071823120117, 'micro_precision': 0.3656249940395355, 'macro_recall': 0.3656249940395355, 'micro_recall': 0.3656249940395355, 'macro_f1': 0.3484755754470825, 'micro_f1': 0.3656249940395355, 'Kappa': 0.2750000059604645, 'MSE': 0.09610182046890259, 'LogLoss': 1.7093902826309204, 'AUNU': 0.8005011160714286, 'micro_ROC-AUC': 0.7856203962053572}, {'macro_accuracy': 0.8132812976837158, 'micro_accuracy': 0.813281238079071, 'macro_precision': 0.29193541407

In [70]:
import pandas as pd

df_metrics = pd.DataFrame(selected_metrics)

correlation_matrix = df_metrics.corr(method='pearson')
print(correlation_matrix)

                 macro_accuracy  micro_accuracy  macro_precision  \
macro_accuracy         1.000000        1.000000         0.941285   
micro_accuracy         1.000000        1.000000         0.941285   
macro_precision        0.941285        0.941285         1.000000   
micro_precision        1.000000        1.000000         0.941285   
macro_recall           1.000000        1.000000         0.941285   
micro_recall           1.000000        1.000000         0.941285   
macro_f1               0.997486        0.997486         0.960984   
micro_f1               1.000000        1.000000         0.941285   
Kappa                  1.000000        1.000000         0.941285   
MSE                   -0.988621       -0.988621        -0.962363   
LogLoss               -0.976367       -0.976367        -0.983911   
AUNU                   0.955634        0.955634         0.982423   
micro_ROC-AUC          0.975257        0.975257         0.981546   

                 micro_precision  macro_recall 